# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [23]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [24]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_BASE_URL = "http://localhost:11434/v1"

In [25]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


API key looks good so far


In [26]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [27]:
def stream_answer(llm, question, model):
    stream = llm.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a supremely helpful expert on every topic. Answer my question"},
            {"role": "user", "content": question}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [28]:
# Get gpt-4o-mini to answer, with streaming
stream_answer(openai, question, MODEL_GPT)

The provided code snippet is using a Python feature called "generator expressions" in combination with the `yield from` statement. Let's break down the code:

1. **Generator Expression:** The part inside the braces `{...}` is a set comprehension. It iterates over each `book` in a collection called `books`, and for each `book`, it retrieves the value associated with the `"author"` key using `book.get("author")`. If the `"author"` key exists and its value is not `None` (or if it is an empty string), that value is included in the resulting set. The condition `if book.get("author")` acts as a filter to ensure that only valid author values are included. 

    - For example, if `books` contained:
      ```python
      books = [
          {"title": "Book 1", "author": "Author A"},
          {"title": "Book 2", "author": None},
          {"title": "Book 3", "author": "Author B"},
          {"title": "Book 4", "author": ""},
          {"title": "Book 5", "author": "Author C"},
      ]
      ```
      The resulting set of authors would be `{"Author A", "Author B", "Author C"}`.

2. **Set:** The braces `{...}` indicate that this is a set comprehension, which means the result is a set containing unique authors. If there are duplicate authors among the books, they will only appear once in the set.

3. **`yield from` Statement:** The `yield from` keyword is used in a generator to yield all values from an iterable (in this case, the set of authors). This means that the function containing this code will yield each author one by one when iterated over.

### Summary
In summary, this line of code collects the unique authors from a list of `books` where the authors are specified (i.e., not `None` or empty) and yields them one by one as needed. This is useful for creating a generator that allows for lazy evaluation and efficient iteration over authors without needing to build and store an entire list of them in memory at once.

In [29]:
# Get Llama 3.2 to answer
stream_answer(ollama, question, MODEL_LLAMA)

This line of code uses a feature of Python 3.10 called **yield from**, along with a **generator expression**, which is a shorthand way to create a generator function.

**Breaking it down:**

* `book.get("author")`: This tries to get the value associated with the key `"author"` from each dictionary (`book`) that you're iterating over.
* `{... for book in books ... }`: This is a **generator expression**, which is similar to a list comprehension, but instead of building a new Python object, it builds an iterator (a generator).
* `yield from {...}`: This is the magic part. When used inside a function or another generator, `yield from` "sub-generates" a sequence yielded by another iterator.

So, putting it all together:

```python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}
```

This code does two main things:

1. **Filtering**: It filters out any dictionaries (`book`) that don't have an `"author"` key.
2. **Yielding values iteratively**: For each remaining dictionary, it yields their corresponding author value as a string (or `None` if the value is missing).

**Why is this useful?**

This code allows you to efficiently iterate over the authors of multiple books at once without having to create intermediate collections or load all book data into memory. It's particularly suitable for large datasets.

Here are some use cases where you might want to use this approach:

* Processing a list of articles with various metadata, like author profiles.
* Retrieving data from an API that returns book objects or documents with complex structures (e.g., author information).

Example usage:
```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Author C"}
]

for author in get_authors(books):
    print(author)
```
Output:
```
Author A
None
Author C
```